In [1]:
# Import necessary libraries
import socket
import time
import re
import json

# Twitch IRC Credentials
NICK = "bobo"
PASS = "oauth:xxxxxxxxxxxxxxxxxxxxxxxxxxx"
CHANNEL_NAME = "pokelawls"

# Socket Server Configuration
SERVER_HOST = "127.0.0.1"
SERVER_PORT = 8081

# Twitch IRC Server Configuration
TWITCH_SERVER = "irc.chat.twitch.tv"
TWITCH_PORT = 6667

# Compile a regular expression to match chat messages from Twitch IRC
CHAT_MSG = re.compile(r"^:\w+!\w+@\w+.tmi.twitch.tv PRIVMSG #\w+ :")

# Function to parse messages from Twitch IRC response
def parse_message(response):
    if "PRIVMSG" not in response:
        return None, None
    
    # Extract username from response
    username = re.search(r"\w+", response).group(0)
    # Extract message from response
    message = CHAT_MSG.sub("", response)
    return username, message.strip()

# Main function to run the server
def run():
    # Create socket for Twitch IRC
    twitch_socket = socket.socket()
    # Connect to Twitch IRC server
    twitch_socket.connect((TWITCH_SERVER, TWITCH_PORT))
    # Send credentials and join channel
    twitch_socket.send(f"PASS {PASS}\n".encode("utf-8"))
    twitch_socket.send(f"NICK {NICK}\n".encode("utf-8"))
    twitch_socket.send(f"JOIN #{CHANNEL_NAME}\n".encode("utf-8"))

    # Create socket for the server
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # Bind the socket to the host and port
    server_socket.bind((SERVER_HOST, SERVER_PORT))
    # Listen for incoming connections
    server_socket.listen(1)
    print(f"Server is listening on {SERVER_HOST}:{SERVER_PORT}")

    # Accept incoming client connections
    client_socket, client_address = server_socket.accept()
    print(f"Client {client_address} connected.")

    try:
        while True:
            # Receive data from Twitch IRC
            response = twitch_socket.recv(1024).decode("utf-8")
            # Respond to PING messages from Twitch IRC
            if response.startswith("PING"):
                twitch_socket.send("PONG\n".encode("utf-8"))
            # Process non-empty response
            elif len(response) > 0:
                username, message = parse_message(response)
                # Check if the response is a chat message
                if CHAT_MSG.match(response):
                    # Create a dictionary with message data
                    msg_data = {
                        "date": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                        "username": username,
                        "message": message
                    }
                    # Send the message data to the client as a JSON string
                    client_socket.send((json.dumps(msg_data) + "\n").encode("utf-8"))

    finally:
        # Close the client and server sockets
        client_socket.close()
        server_socket.close()

# Run the server
if __name__ == "__main__":
    try:
        run()
    except KeyboardInterrupt:
        print("\nServer stopped.")


Server is listening on 127.0.0.1:8081
Client ('127.0.0.1', 43164) connected.
